<a href="https://colab.research.google.com/github/andersonsps-dsa/webscraping_futbr/blob/main/ws_S.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Install Library

In [ ]:
pip install -q requests pandas

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, mean_absolute_error

import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import numpy as np
import requests
from google.colab import sheets

# headers serve para simular um navegador
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/137.0.0.0 Safari/537.36"
}

#### Dataframe

In [ ]:
# @title Dataframe
all_data_1 = []

for rodada in range(39):
    url_1 = f'https://www.sofascore.com/api/v1/unique-tournament/325/season/72034/events/round/{rodada}'
    response_1 = requests.get(url_1, headers=headers)

    if response_1.status_code == 200:
        data_1 = response_1.json()

        if 'events' in data_1:
            for event_1 in data_1['events']:
                if isinstance(event_1, dict):

                    id_jogo_1 = event_1.get('id', 0)
                    tempo_seg = event_1.get('startTimestamp', 0)
                    rodada = event_1.get('roundInfo', {}).get('round')
                    status_jogo_1 = event_1.get('status', {}).get('type')

                    HOME_team_1 = event_1.get('homeTeam', {}).get('name',0)
                    HOME_team_id_1 = event_1.get('homeTeam').get('id',0)

                    AWAY_team_1 = event_1.get('awayTeam', {}).get('name', 0)
                    AWAY_team_id_1 = event_1.get('awayTeam', {}).get('id',0)

                    for IDjogo_0 in [id_jogo_1]:
                      url_2 = f'https://www.sofascore.com/api/v1/event/{IDjogo_0}/incidents'
                      response_2 = requests.get(url_2, headers=headers)
                      if response_2.status_code == 200:
                        data_2 = response_2.json()
                        if data_2['incidents']:
                          HOME_score_2 = data_2['incidents'][0].get('homeScore', 0) if 'incidents' in data_2 else 0
                          AWAY_score_2 = data_2['incidents'][0].get('awayScore', 0) if 'incidents' in data_2 else 0
                        else:
                          HOME_score_2 = 0
                          AWAY_score_2 = 0

                    Finalizacoes_H_ALL = Finalizacoes_A_ALL = 0
                    Finalizacoes_H_1ST = Finalizacoes_A_1ST = 0
                    Finalizacoes_H_2ND = Finalizacoes_A_2ND = 0
                    Cantos_H_ALL = Cantos_A_ALL = 0
                    Cantos_H_1ST = Cantos_A_1ST = 0
                    Cantos_H_2ND = Cantos_A_2ND = 0
                    Faltas_H_ALL = Faltas_A_ALL = 0
                    Faltas_H_1ST = Faltas_A_1ST = 0
                    Faltas_H_2ND = Faltas_A_2ND = 0
                    Faltas_Tiro_Direto_H = Faltas_Tiro_Direto_A = 0
                    YELLOWcard_H_ALL = YELLOWcard_A_ALL = 0
                    YELLOWcard_H_1ST = YELLOWcard_A_1ST = 0
                    YELLOWcard_H_2ND = YELLOWcard_A_2ND = 0
                    REDcard_H_ALL = REDcard_A_ALL = 0
                    REDcard_H_1ST = REDcard_A_1ST = 0
                    REDcard_H_2ND = REDcard_A_2ND = 0

                    for IDjogo_1 in [id_jogo_1]:
                        url_3 = f'https://www.sofascore.com/api/v1/event/{IDjogo_1}/statistics'
                        response_3 = requests.get(url_3, headers=headers)

                        if response_3.status_code == 200:
                            data_3 = response_3.json()

                            if 'statistics' in data_3:
                                for stats_3 in data_3['statistics']:
                                    if isinstance(stats_3, dict):
                                        period = stats_3.get('period')
                                        for indg in range(len(stats_3['groups'])):
                                            for ind in range(len(stats_3['groups'][indg]['statisticsItems'])):
                                                if ind < len(stats_3['groups'][indg]['statisticsItems']):
                                                    stat_item = stats_3['groups'][indg]['statisticsItems'][ind]
                                                    key = stat_item.get('key')
                                                    home_value = stat_item.get('home', 0)
                                                    away_value = stat_item.get('away', 0)

                                                    if key == 'totalShotsOnGoal':
                                                        if period == 'ALL':
                                                            Finalizacoes_H_ALL = home_value
                                                            Finalizacoes_A_ALL = away_value
                                                        elif period == '1ST':
                                                            Finalizacoes_H_1ST = home_value
                                                            Finalizacoes_A_1ST = away_value
                                                        elif period == '2ND':
                                                            Finalizacoes_H_2ND = home_value
                                                            Finalizacoes_A_2ND = away_value

                                                    elif key == 'cornerKicks':
                                                        if period == 'ALL':
                                                            Cantos_H_ALL = home_value
                                                            Cantos_A_ALL = away_value
                                                        elif period == '1ST':
                                                            Cantos_H_1ST = home_value
                                                            Cantos_A_1ST = away_value
                                                        elif period == '2ND':
                                                            Cantos_H_2ND = home_value
                                                            Cantos_A_2ND = away_value

                                                    elif key == 'fouls':
                                                        if period == 'ALL':
                                                            Faltas_H_ALL = home_value
                                                            Faltas_A_ALL = away_value

                                                    elif key == 'freeKicks':
                                                        if period == 'ALL':
                                                            Faltas_TiroDireto_H_ALL = home_value
                                                            Faltas_TiroDireto_A_ALL = away_value
                                                        elif period == '1ST':
                                                            Faltas_TiroDireto_H_1ST = home_value
                                                            Faltas_TiroDireto_A_1ST = away_value
                                                        elif period == '2ND':
                                                            Faltas_TiroDireto_H_2ND = home_value
                                                            Faltas_TiroDireto_A_2ND = away_value

                                                    elif key == 'yellowCards':
                                                        if period == 'ALL':
                                                            YELLOWcard_H_ALL = home_value
                                                            YELLOWcard_A_ALL = away_value
                                                        elif period == '1ST':
                                                            YELLOWcard_H_1ST = home_value
                                                            YELLOWcard_A_1ST = away_value
                                                        elif period == '2ND':
                                                            YELLOWcard_H_2ND = home_value
                                                            YELLOWcard_A_2ND = away_value

                                                    elif key == 'redCards':
                                                        if period == 'ALL':
                                                            REDcard_H_ALL = home_value
                                                            REDcard_A_ALL = away_value
                                                        elif period == '1ST':
                                                            REDcard_H_1ST = home_value
                                                            REDcard_A_1ST = away_value
                                                        elif period == '2ND':
                                                            REDcard_H_2ND = home_value
                                                            REDcard_A_2ND = away_value

                    all_data_1.append({
                        'id_jogo': id_jogo_1,
                        'tempo_seg': tempo_seg,
                        'Rodada': rodada,
                        'Status Jogo': status_jogo_1,
                        'Team_H': HOME_team_1,
                        'Team_H_ID': HOME_team_id_1,
                        'Gols_H_ALL': HOME_score_2,
                        'Team_A': AWAY_team_1,
                        'Team_A_ID': AWAY_team_id_1,
                        'Gols_A_ALL': AWAY_score_2,
                        'Finalizacoes_H_ALL': Finalizacoes_H_ALL,
                        'Finalizacoes_A_ALL': Finalizacoes_A_ALL,
                        'Finalizacoes_H_1ST': Finalizacoes_H_1ST,
                        'Finalizacoes_A_1ST': Finalizacoes_A_1ST,
                        'Finalizacoes_H_2ND': Finalizacoes_H_2ND,
                        'Finalizacoes_A_2ND': Finalizacoes_A_2ND,
                        'Cantos_H_ALL': Cantos_H_ALL,
                        'Cantos_A_ALL': Cantos_A_ALL,
                        'Cantos_H_1ST': Cantos_H_1ST,
                        'Cantos_A_1ST': Cantos_A_1ST,
                        'Cantos_H_2ND': Cantos_H_2ND,
                        'Cantos_A_2ND': Cantos_A_2ND,
                        'Faltas_H_ALL': Faltas_H_ALL,
                        'Faltas_A_ALL': Faltas_A_ALL,
                        'Faltas_TiroDireto_H_ALL': Faltas_TiroDireto_H_ALL,
                        'Faltas_TiroDireto_A_ALL': Faltas_TiroDireto_A_ALL,
                        'Faltas_TiroDireto_H_1ST':Faltas_TiroDireto_H_1ST,
                        'Faltas_TiroDireto_A_1ST':Faltas_TiroDireto_A_1ST,
                        'Faltas_TiroDireto_H_2ND':Faltas_TiroDireto_H_2ND,
                        'Faltas_TiroDireto_A_2ND':Faltas_TiroDireto_A_2ND,
                        'YELLOWCard_H_ALL': YELLOWcard_H_ALL,
                        'YELLOWCard_A_ALL': YELLOWcard_A_ALL,
                        'YELLOWCard_H_1ST': YELLOWcard_H_1ST,
                        'YELLOWCard_A_1ST': YELLOWcard_A_1ST,
                        'YELLOWCard_H_2ND': YELLOWcard_H_2ND,
                        'YELLOWCard_A_2ND': YELLOWcard_A_2ND,
                        'REDCard_H_ALL': REDcard_H_ALL,
                        'REDCard_A_ALL': REDcard_A_ALL,
                        'REDCard_H_1ST': REDcard_H_1ST,
                        'REDCard_A_1ST': REDcard_A_1ST,
                        'REDCard_H_2ND': REDcard_H_2ND,
                        'REDCard_A_2ND': REDcard_A_2ND
                      })

base = pd.DataFrame(all_data_1)
base_1 = base.copy()
base_1 = base.loc[base['Status Jogo'] == 'finished']
base_1.drop_duplicates(inplace=True)
base_1.reset_index(drop=True, inplace=True)

KeyError: 'Status Jogo'